In [1]:
from atm import ATM

atm = ATM()

In [2]:
df_train = spark.read.format('csv').options(header=True).load('/titanic/train.csv')
df_test = spark.read.format('csv').options(header=True).load('/titanic/test.csv')

In [3]:
# eliminamos las variables que no tienen valor predictivo

from functools import reduce
from pyspark.sql import DataFrame

train = reduce(DataFrame.drop, ['PassengerId','Name','Ticket', 'Cabin'], df_train)
test = reduce(DataFrame.drop, ['PassengerId','Name','Ticket', 'Cabin'], df_test)

In [4]:
# rellenamos los nulos con los valores medios estableciendo el tipo correcto de cada variable 

from pyspark.sql.functions import avg
from pyspark.sql.types import IntegerType, DoubleType
import pandas as pd
import numpy as np

def fill_with_mean(df, include=set()): 
  stats = df.agg(*(
    avg(c).alias(c) for c in include
  ))
  return df.na.fill(stats.first().asDict())

def fill_and_cast(df, survived):
  df = fill_with_mean(df, ["Age", "SibSp", "Parch", "Fare"])
  df = df.withColumn("Age", df["Age"].cast(IntegerType()))
  df = df.withColumn("SibSp", df["SibSp"].cast(IntegerType()))
  df = df.withColumn("Parch", df["Parch"].cast(IntegerType()))
  df = df.withColumn("Fare", df["Fare"].cast(DoubleType()))
  df = df.na.fill({'Embarked': 'S'})
  df = df.withColumn("Pclass",df["Pclass"].cast(IntegerType()))
  if survived == True:
      df = df.withColumn("Survived",df["Survived"].cast(IntegerType()))
  return df

train = fill_and_cast(train, True)
test = fill_and_cast(test, False)

display(train)

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22,1,0,7.25,S
1,1,female,38,1,0,71.2833,C
1,3,female,26,0,0,7.925,S
1,1,female,35,1,0,53.1,S
0,3,male,35,0,0,8.05,S
0,3,male,29,0,0,8.4583,Q
0,1,male,54,0,0,51.8625,S
0,3,male,2,3,1,21.075,S
1,3,female,27,0,2,11.1333,S
1,2,female,14,1,0,30.0708,C


In [5]:
train.printSchema()

root
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Sex: string (nullable = true)
-- Age: integer (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Fare: double (nullable = true)
-- Embarked: string (nullable = false)

In [6]:
train.withColumnRenamed("Survived","class").toPandas().to_csv('/dbfs/titanic/train-atm.csv', index = False)

In [7]:
results = atm.run(train_path='/dbfs/titanic/train-atm.csv')

Processing dataset /dbfs/titanic/train-atm.csv
 0%| | 0/100 [00:00<?, ?it/s] 1%|1 | 1/100 [00:00<00:33, 2.98it/s] 2%|2 | 2/100 [00:00<00:35, 2.76it/s] 3%|3 | 3/100 [00:01<00:34, 2.79it/s] 4%|4 | 4/100 [00:01<00:35, 2.74it/s] 5%|5 | 5/100 [00:01<00:34, 2.73it/s] 6%|6 | 6/100 [00:02<00:34, 2.73it/s] 7%|7 | 7/100 [00:02<00:35, 2.63it/s] 8%|8 | 8/100 [00:03<00:35, 2.56it/s] 9%|9 | 9/100 [00:03<00:34, 2.64it/s] 10%|# | 10/100 [00:03<00:33, 2.70it/s] 11%|#1 | 11/100 [00:04<00:33, 2.66it/s] 12%|#2 | 12/100 [00:04<00:32, 2.68it/s] 13%|#3 | 13/100 [00:04<00:33, 2.63it/s] 14%|#4 | 14/100 [00:05<00:33, 2.56it/s] 15%|#5 | 15/100 [00:05<00:33, 2.51it/s] 16%|#6 | 16/100 [00:06<00:32, 2.59it/s] 17%|#7 | 17/100 [00:06<00:32, 2.58it/s] 18%|#8 | 18/100 [00:06<00:30, 2.65it/s] 19%|#9 | 19/100 [00:07<00:30, 2.66it/s] 20%|## | 20/100 [00:07<00:31, 2.54it/s] 21%|##1 | 21/100 [00:08<00:30, 2.56it/s] 22%|##2 | 22/100 [00:08<00:29, 2.61it/s] 23%|##3 | 23/100 [00:08<00:31, 2.45it/s] 24%|##4 | 24/100 [00:09<00:30, 2.53it/s] 25%|##5 | 25/100 [00:10<00:40, 1.83it/s] 26%|##6 | 26/100 [00:10<00:36, 2.01it/s] 27%|##7 | 27/100 [00:10<00:33, 2.17it/s] 28%|##8 | 28/100 [00:11<00:31, 2.31it/s] 29%|##9 | 29/100 [00:11<00:29, 2.38it/s] 30%|### | 30/100 [00:12<00:29, 2.38it/s] 31%|###1 | 31/100 [00:12<00:29, 2.37it/s] 32%|###2 | 32/100 [00:12<00:28, 2.35it/s] 33%|###3 | 33/100 [00:13<00:27, 2.40it/s] 34%|###4 | 34/100 [00:13<00:26, 2.51it/s] 35%|###5 | 35/100 [00:14<00:36, 1.77it/s] 36%|###6 | 36/100 [00:15<00:32, 1.95it/s] 37%|###7 | 37/100 [00:15<00:30, 2.09it/s] 38%|###8 | 38/100 [00:15<00:27, 2.23it/s] 39%|###9 | 39/100 [00:16<00:25, 2.35it/s] 40%|#### | 40/100 [00:16<00:24, 2.41it/s] 41%|####1 | 41/100 [00:16<00:24, 2.43it/s] 42%|####2 | 42/100 [00:17<00:23, 2.49it/s] 43%|####3 | 43/100 [00:17<00:23, 2.46it/s] 44%|####4 | 44/100 [00:18<00:21, 2.55it/s] 45%|####5 | 45/100 [00:18<00:21, 2.56it/s] 46%|####6 | 46/100 [00:18<00:20, 2.63it/s] 47%|####6 | 47/100 [00:19<00:20, 2.57it/s] 48%|####8 | 48/100 [00:19<00:20, 2.58it/s] 49%|####9 | 49/100 [00:20<00:20, 2.54it/s] 50%|##### | 50/100 [00:20<00:19, 2.62it/s] 51%|#####1 | 51/100 [00:20<00:19, 2.46it/s] 52%|#####2 | 52/100 [00:21<00:19, 2.45it/s] 53%|#####3 | 53/100 [00:21<00:18, 2.51it/s] 54%|#####4 | 54/100 [00:22<00:17, 2.59it/s] 55%|#####5 | 55/100 [00:22<00:18, 2.49it/s] 56%|#####6 | 56/100 [00:22<00:17, 2.51it/s] 57%|#####6 | 57/100 [00:23<00:17, 2.52it/s] 58%|#####8 | 58/100 [00:23<00:16, 2.53it/s] 59%|#####8 | 59/100 [00:24<00:16, 2.49it/s] 60%|###### | 60/100 [00:24<00:16, 2.42it/s] 61%|######1 | 61/100 [00:25<00:21, 1.81it/s] 62%|######2 | 62/100 [00:25<00:19, 1.96it/s] 63%|######3 | 63/100 [00:26<00:17, 2.11it/s] 64%|######4 | 64/100 [00:26<00:20, 1.79it/s] 65%|######5 | 65/100 [00:27<00:18, 1.90it/s] 66%|######6 | 66/100 [00:27<00:16, 2.06it/s] 67%|######7 | 67/100 [00:28<00:15, 2.11it/s] 68%|######8 | 68/100 [00:28<00:14, 2.21it/s] 69%|######9 | 69/100 [00:29<00:13, 2.28it/s] 70%|####### | 70/100 [00:29<00:12, 2.43it/s] 71%|#######1 | 71/100 [00:29<00:11, 2.42it/s] 72%|#######2 | 72/100 [00:30<00:11, 2.38it/s] 73%|#######3 | 73/100 [00:30<00:10, 2.50it/s] 74%|#######4 | 74/100 [00:30<00:10, 2.47it/s] 75%|#######5 | 75/100 [00:31<00:10, 2.48it/s] 76%|#######6 | 76/100 [00:31<00:09, 2.46it/s] 77%|#######7 | 77/100 [00:32<00:09, 2.47it/s] 78%|#######8 | 78/100 [00:32<00:08, 2.48it/s] 79%|#######9 | 79/100 [00:32<00:08, 2.59it/s] 80%|######## | 80/100 [00:33<00:07, 2.54it/s] 81%|########1 | 81/100 [00:33<00:07, 2.54it/s] 82%|########2 | 82/100 [00:34<00:07, 2.56it/s] 83%|########2 | 83/100 [00:34<00:06, 2.62it/s] 84%|########4 | 84/100 [00:35<00:08, 1.84it/s] 85%|########5 | 85/100 [00:35<00:07, 2.03it/s] 86%|########6 | 86/100 [00:36<00:07, 1.94it/s] 87%|########7 | 87/100 [00:36<00:06, 2.08it/s] 88%|########8 | 88/100 [00:37<00:05, 2.17it/s] 89%|########9 | 89/100 [00:37<00:04, 2.34it/s] 90%|######### | 90/100 [00:

In [8]:
results.describe()

Datarun 1 summary:
	Dataset: '/dbfs/titanic/train-atm.csv'
	Column Name: 'class'
	Judgment Metric: 'f1'
	Classifiers Tested: 100
	Elapsed Time: 0:00:42.358296

In [9]:
results.get_best_classifier()

Out[9]: Classifier id: 15
Classifier type: knn
Params chosen: 
	n_neighbors: 10
	leaf_size: 33
	p: 2
	weights: uniform
	algorithm: ball_tree
	metric: minkowski
	_scale: True
Cross Validation Score: 0.762 +- 0.028
Test Score: 0.699

In [10]:
results.export_best_classifier('/dbfs/titanic/atm.pkl', force=True)

Classifier 15 saved as /dbfs/titanic/atm.pkl

In [11]:
from atm import Model

model = Model.load('/dbfs/titanic/atm.pkl')

In [12]:
predictions = model.predict(test.toPandas())

In [13]:
preds = pd.DataFrame({'PassengerId' : range(892,1310), 'Survived' : pd.Series(predictions)})
preds.to_csv('/dbfs/titanic/results-atm.csv', index=False)